In [17]:
import pandas as pd
import re
import numpy as np
import requests

In [23]:
%load_ext autoreload
%autoreload 2


In [24]:
tracks=pd.read_csv('data/spotify_2017.csv')

In [25]:
tracks.head(3)

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0,4.0
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,-4.328,1.0,0.1200,0.229,0.0,0.0924,0.813,88.931,228827.0,4.0
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,-4.757,1.0,0.1700,0.209,0.0,0.1120,0.846,177.833,228200.0,4.0


In [26]:
tracks_sub=tracks[['name','artists']]

In [28]:
from music import fetch_lyrics

In [29]:
fetch_lyrics('The Beatles', 'Yesterday')

"Yesterday all my troubles seemed so far away.\nNow it looks as though they're here to stay.\nOh, I believe in yesterday.\n\nSuddenly, I'm not half the man I used to be.\nThere's a shadow hanging over me.\nOh, yesterday came suddenly.\n\nWhy she had to go?\nI don't know, she wouldn't say.\nI said something wrong.\nNow I long for yesterday.\n\nYesterday love was such an easy game to play.\nNow I need a place to hide away.\nOh, I believe in yesterday.\n\nWhy she had to go?\nI don't know, she wouldn't say.\nI said something wrong.\nNow I long for yesterday.\n\nYesterday love was such an easy game to play.\nNow I need a place to hide away.\nOh, I believe in yesterday.\n\nMm mm mm mm mm mm mm."

In [30]:
fetch_lyrics('The Beatles', 'Today')

''

In [32]:
tracks["lyrics"]=''

In [34]:
%%time
for (index, row) in tracks.head(3).iterrows():
    print(f"fetching {row['artists']} - {row['name']}")
    tracks.loc[index,'lyrics'] = fetch_lyrics(row['artists'], row['name'])

fetching Ed Sheeran - Shape of You
fetching Luis Fonsi - Despacito - Remix
fetching Luis Fonsi - Despacito (Featuring Daddy Yankee)
CPU times: user 92.4 ms, sys: 11.1 ms, total: 103 ms
Wall time: 1.86 s


In [36]:
print(tracks.loc[0, 'lyrics'][:100])

The club isn't the best place to find a lover
So the bar is where I go (mmmm)
Me and my friends at t


## Google Big Query

In [42]:
import pandas_gbq

In [38]:
project_id="povertymapper"

In [39]:
sql = """
SELECT faa_identifier, name, longitude, latitude, airport_type, service_city, country
FROM `bigquery-public-data.faa.us_airports`
WHERE airport_use = 'Public'
"""

In [43]:
airport_df=pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████████████████| 5368/5368 [00:01<00:00, 4683.98rows/s]


In [44]:
airport_df.head(3)

,faa_identifier,name,longitude,latitude,airport_type,service_city,country
0,2A4,Vor Lake Waterlane,-151.494167,66.887445,Seaport,Bettles,United States
1,5C5,El Capitan Lodge,-133.253391,55.958593,Seaport,Craig,United States
2,3A3,Seymour Lake Spb,-149.665593,61.613463,Seaport,Wasilla,United States


## Scraping

In [50]:
from bs4 import BeautifulSoup

In [48]:
link='https://www.imdb.com/list/ls055386972/'
response=requests.get(link, headers={'Accept-Language':'en-US'})

In [51]:
soup = BeautifulSoup(response.content, 'html.parser')

In [61]:
movies=[]

for movie in soup.find_all("div", class_="lister-item-content"):
    title=movie.find('h3').find('a').string
    year=movie.find(class_="lister-item-year").string
    duration=movie.find(class_="runtime").string
    movies.append({'title':title, 'year':year, 'duration':duration})
    

In [62]:
movies

[{'title': 'The Godfather', 'year': '(1972)', 'duration': '175 min'},
 {'title': "Schindler's List", 'year': '(1993)', 'duration': '195 min'},
 {'title': '12 Angry Men', 'year': '(1957)', 'duration': '96 min'},
 {'title': 'Life Is Beautiful', 'year': '(1997)', 'duration': '116 min'},
 {'title': 'The Good, the Bad and the Ugly',
  'year': '(1966)',
  'duration': '178 min'},
 {'title': 'The Shawshank Redemption',
  'year': '(1994)',
  'duration': '142 min'},
 {'title': 'The Pursuit of Happyness',
  'year': '(2006)',
  'duration': '117 min'},
 {'title': 'Seven Samurai', 'year': '(1954)', 'duration': '207 min'},
 {'title': 'The Intouchables', 'year': '(2011)', 'duration': '112 min'},
 {'title': 'Central Station', 'year': '(1998)', 'duration': '110 min'},
 {'title': 'Requiem for a Dream', 'year': '(2000)', 'duration': '102 min'},
 {'title': 'A Beautiful Mind', 'year': '(2001)', 'duration': '135 min'},
 {'title': "Hachi: A Dog's Tale", 'year': '(2009)', 'duration': '93 min'},
 {'title': 'Tak

## From List to Dataframe

In [64]:
movies_df=pd.DataFrame(movies)

In [65]:
movies_df.head(3)

,title,year,duration
0,The Godfather,(1972),175 min
1,Schindler's List,(1993),195 min
2,12 Angry Men,(1957),96 min
